In [ ]:
#imports
import sys
import pandas as pd
import numpy as np
import os
import random
import logging
from livelossplot import PlotLosses
import pickle
import torch
import monai
from monai.data import DataLoader
from monai.transforms import (
    AddChanneld,
    CenterSpatialCropd,
    Compose,
    Resized,
    RandSpatialCropd,
    ScaleIntensityd,
    ToTensord,
    LoadImaged,
    Identityd,
)
from monai.utils import InterpolateMode
import nibabel as nib
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [ ]:
#hyperparameters which were selected during hyperparameter tuning
lr=1e-4
opt="none"
strategy="adam"
epoch=84

In [ ]:
#definitions of path
MODEL_DIR = os.path.join("./SEResNet/")
path_train_data=os.path.join("../../data/trainValid_DL.csv")
mapping_ML_DL=os.path.join("../../additional_data/Mapping_DKT_Regions_Deep_ML_new.csv")
aspects_filename=os.path.join("../../additional_data/aspects05_new.pkl")
freesurfer_mapping_filename=os.path.join("../../additional_data/freesurferMappingReduced.csv")
gradCAM_image_directory=os.path.join("./SEResNet/GradCAM/")
gradCAM_save_individual_results_path=os.path.join("./SEResNet/GradCAM/GradCAM_individual.csv")
gradCAM_save_global_results_path=os.path.join("./SEResNet/GradCAM/GradCAM_global.csv")

In [ ]:
#if model directory not exists create GradCAM directory
if not os.path.exists(gradCAM_image_directory):
    os.makedirs(gradCAM_image_directory)

In [ ]:
BATCH_SIZE=1

In [ ]:
#load training dataset
trainValidMerged=pd.read_csv(path_train_data,index_col="PTID")

In [ ]:
#load data augmentations
validation_transforms = Compose(
        [
            LoadImaged(keys=["img","segmentation"]),
            AddChanneld(keys=["img","segmentation"]),
            ScaleIntensityd(keys=["img"]),
            Resized(keys=["img"],spatial_size=(256,256,256)),
            Resized(keys=["segmentation"],spatial_size=(256,256,256),mode=InterpolateMode.NEAREST),
            CenterSpatialCropd(keys=["img","segmentation"],roi_size=(224,224,224)),
            ToTensord(keys=["img","segmentation"]),
        ]
    )


In [ ]:
#define function to set seeds for reproducibility
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
#reformat training dataset to pytorch
Y_train=pd.get_dummies(trainValidMerged.DX,drop_first=True).to_numpy().squeeze()
Y_train=Y_train.tolist()
trainDSNew = [{"PTID":ptid,"img": img, "label": label,"segmentation":segmentation} for ptid,img, label,segmentation in zip(trainValidMerged.index,trainValidMerged.filename, Y_train,trainValidMerged.filenameSeg)]
set_seed(123)
train_ds = monai.data.Dataset(data=trainDSNew, transform=validation_transforms)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, pin_memory=torch.cuda.is_available())

In [ ]:
#choose cuda as the device if it is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#load DL model using monai
model = monai.networks.nets.SEResNet152(num_classes=2,spatial_dims=3, in_channels=1)
#load final model weights
PATH=MODEL_DIR+"model_"+str(opt)+"_"+str(lr)+"_"+str(strategy)+"_"+str(epoch)+"_final_model_polyak_averaged.pth"
model.load_state_dict(torch.load(PATH))  # Choose whatever GPU device number you want
model=model.to(device)

In [ ]:
#initalize GradCAM explainer
cam = monai.visualize.GradCAM(
    nn_module=model, target_layers="layer4"
)

In [ ]:
#load aspects
with open(aspects_filename, 'rb') as f:
    aspects = pickle.load(f)
#load ML/DL feature mapping
mapping=pd.read_csv(mapping_ML_DL)

In [ ]:
#load freesurfer segmentation mapping
freesurferMapping=pd.read_table(freesurfer_mapping_filename,sep=",")

In [ ]:
#mapping between brain regions and freesurfer segmentation regions
mergedDF=pd.merge(mapping,freesurferMapping,how="outer",right_on="brain region",left_on="feature_Deep")
#map aspects to freesurfer segmentations
for key in aspects:
    aspects[key]=mergedDF[mergedDF.feature_ML.isin(aspects[key])].ID.tolist()

#identify brain regions not available in ML models
notInML=freesurferMapping[~freesurferMapping["brain region"].isin(mapping.feature_Deep)]

In [ ]:
#add brain regions without ML volumes to aspects
for index, row in notInML.iterrows():
    aspects[row["brain region"]]=[row["ID"]]       

In [ ]:
#initialize dataframe to save activated brain regions of GradCAM map
column_names = ["PTID","label","pred"]+list(aspects.keys())
df = pd.DataFrame(columns = column_names)

In [ ]:
indexi=0
#change model to evaluation model
model.eval()
#iterate over training dataset
for val_data in train_loader:
    indexi+=1
    #load input scan, label and freesurfer segmentation mask of training data sample
    img=val_data["img"].cuda()
    label=val_data["label"].numpy()[0]
    seg=val_data["segmentation"].cuda()
    #calculate model prediction for training data sample
    pred=model(img).argmax(dim=1).cpu().detach().numpy()[0]
    #compute GradCAM result for training sample
    result = cam(x=img)
    #invert GradCAM result https://docs.monai.io/en/stable/visualize.html (last accessed: 2024-01-18)
    result=result*(-1)+1
    #save GradCAM map to specified directory
    result_test=np.squeeze(result.cpu().numpy())
    result_image = nib.Nifti1Image(result_test, affine=np.eye(4))
    nib.save(result_image, gradCAM_image_directory+"GradCAM_PTID_"+val_data["PTID"][0]+".nii.gz")  
    #flatten segmentation
    seg_flattened=seg.flatten()
    #generate new segmentation based on aspects
    seg_Flattened_new=torch.zeros_like(seg_flattened)
    i=1
    for aspect in aspects:
        for value in aspects[aspect]:
            seg_Flattened_new[seg_flattened==value]=i
        i+=1
    #flatten GradCAM image
    res_flattened=result.flatten()
    #identify segmentation labels and counts
    colors, counts = np.unique(seg_Flattened_new.cpu().detach().numpy(), axis=0, return_counts=True)
    #initalize array to save summed values of GradCAM scores per aspect
    summedValues=[0]*counts.shape[0]
    summedValues=np.asarray(summedValues)
    #calculate summed values of GradCAM scores
    j=0
    for i in colors:
        summedValues[j]=res_flattened[(seg_Flattened_new==i).cpu().numpy()].sum()
        j+=1
    #calculate mean values of GradCAM scores
    meanValues=summedValues/counts
    #insert 0 values for aspects which are not segmented in the image
    for i in range(1,len(aspects)+1):
        if(~np.isin(i,colors)):
            colors=np.insert(colors,i-1,i)
            meanValues=np.insert(meanValues,i-1,0)
            summedValues=np.insert(summedValues,i-1,0)
            counts=np.insert(counts,i-1,0)
    #save mean GradCAM scores for all aspects
    d = {'ID': colors, 'meanValues': meanValues,'summedValues':summedValues, "counts":counts}
    dfSub = pd.DataFrame(data=d)
    column_names = ["PTID","label","pred"]+list(aspects.keys())
    values=[val_data["PTID"][0],label,pred]+ meanValues.tolist()
    df2 = pd.DataFrame([values],columns = column_names)
    df = pd.concat([df,df2],ignore_index=True)

In [ ]:
#save results at subject level
df.to_csv(gradCAM_save_individual_results_path)

In [ ]:
#save global results
df=df.drop(["PTID"],axis=1)
d = {'feature': df.sum().index.tolist(), 'GradCAMImportance': df.sum().tolist()}
resultsSum=pd.DataFrame(data=d)

In [ ]:
resultsSum.to_csv(gradCAM_save_global_results_path)